In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import altair as alt
from math import sqrt
from scipy.stats import pearsonr, rankdata
from pandas._testing import assert_series_equal

import pandamonium

pd.set_option("precision", 10)

In [ ]:
pd.__version__

In [ ]:
data = pd.read_json("https://raw.githubusercontent.com/vega/vega-datasets/master/data/penguins.json")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data = data.clean.column_names()

In [ ]:
data.head()

# Pearson's correlation

In [ ]:
def pearson_corr(x, y):
    mean_x = x.mean()
    mean_y = y.mean()

    xm = x - mean_x
    ym = y - mean_y

    numerator = (xm * ym).sum()
    denominator = sqrt((xm ** 2).sum()) * sqrt((ym ** 2).sum())

    return numerator / denominator

In [ ]:
pearson_manual = data.corr(method=pearson_corr)
pearson_manual

In [ ]:
pearson = data.corr(method="pearson")
pearson

In [ ]:
pearson.compare(pearson_manual)

In [ ]:
data_nan = data.dropna(subset=["beak_length_mm", "beak_depth_mm"])

round(pearsonr(data_nan["beak_length_mm"], data_nan["beak_depth_mm"])[0], 10)

# Spearman's rank correlation

The Pearson correlation coefficient between the rank variables.

In [ ]:
def compare_rank_data_methods(data, col, method="first"):
    df_compare = pd.DataFrame()

    data_nan = data.dropna(subset=[col])

    df_compare["pandas"] = data_nan[col].rank(
        method="first" if method == "ordinal" else method
    )
    
    df_compare["scipy"] = rankdata(
        data_nan[col], method="ordinal" if method == "first" else method
    )

    assert_series_equal(
        df_compare["pandas"], df_compare["scipy"], check_dtype=False, check_names=False
    )

    return df_compare

In [ ]:
compare_rank_data_methods(data, "beak_length_mm").head(10)